<a href="https://colab.research.google.com/github/fernandoariel/Diplomatura/blob/master/TP_Supervisado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import os

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score

from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV


# **Aprendizaje Supervisado**

**Grupo 14** 
- Fernando Vázquez
- Matias Ramos
- Lucas Lascano
- Ariel Rubio

In [2]:
from google.colab import drive 
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
df=pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/train.csv')
df.sample(10) #le damos un vistaso a los datos

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
181008,33,76390,Wednesday,8.563120e+10,-1,DSD GROCERY,3467.0
395901,40,167337,Thursday,7.874208e+09,2,BAKERY,6715.0
248046,18,104264,Sunday,8.054702e+10,1,JEWELRY AND SUNGLASSES,6812.0
351728,25,148462,Sunday,3.426444e+09,1,COOK AND DINE,579.0
311155,36,131943,Friday,3.700020e+09,1,PERSONAL CARE,203.0
318218,39,135013,Friday,7.952321e+10,1,"GIRLS WEAR, 4-6X AND 7-14",363.0
149739,39,62751,Sunday,4.145811e+09,1,FROZEN FOODS,2006.0
171791,40,72261,Tuesday,2.100005e+09,1,DAIRY,7955.0
43452,37,17440,Sunday,2.840024e+09,1,DSD GROCERY,4402.0
252945,40,106013,Sunday,8.154730e+10,2,DAIRY,7978.0


In [4]:
def transform_data(train_data_fname, test_data_fname):
    df_train = pd.read_csv(train_data_fname)
    df_train['is_train_set'] = 1
    df_test = pd.read_csv(test_data_fname)
    df_test['is_train_set'] = 0

    # we  get the TripType for the train set. To do that, we group by VisitNumber and
    # then we get the max (or min or avg)
    y = df_train.groupby(["VisitNumber", "Weekday"], as_index=False).max().TripType

    # we remove the TripType now, and concat training and testing data
    # the concat is done so that we have the same columns for both datasets
    # after one-hot encoding
    df_train = df_train.drop("TripType", axis=1)
    df = pd.concat([df_train, df_test])
    
    # the next three operations are the ones we have just presented in the previous lines
    
    # drop the columns we won't use (it may be good to use them somehow)
    df = df.drop(["Upc", "FinelineNumber"], axis=1)

    # one-hot encoding for the DepartmentDescription
    df = pd.get_dummies(df, columns=["DepartmentDescription"], dummy_na=True)

    # now we add the groupby values
    df = df.groupby(["VisitNumber", "Weekday"], as_index=False).sum()
    
    # finally, we do one-hot encoding for the Weekday
    df = pd.get_dummies(df, columns=["Weekday"], dummy_na=True)

       

    # get train and test back
    df_train = df[df.is_train_set != 0]
    df_test = df[df.is_train_set == 0]
    
    X = df_train.drop(["is_train_set"], axis=1)
    yy = None
    XX = df_test.drop(["is_train_set"], axis=1)

    return X, y, XX, yy

In [5]:
X, y, XX, yy = transform_data("/content/gdrive/My Drive/Colab Notebooks/train.csv", "/content/gdrive/My Drive/Colab Notebooks/test.csv")

In [6]:
# split training dataset into train and "validation" 
# (we won't be using validation set in this example, because of the cross-validation;
# but it could be useful for you depending on your approach)
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

In [7]:
# results dataframe is used to store the computed results
results = pd.DataFrame(columns=('clf', 'best_acc'))

# **Random Forest con RandomizedSearchCV**

In [8]:
param_distributions = {'criterion': ['gini', 'entropy'], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 6], 'n_estimators': [30,60], 'max_features':['sqrt','log2',6]}
forest_clf=RandomizedSearchCV( RandomForestClassifier(random_state=1) ,param_distributions, n_iter=10, scoring='accuracy', cv=3)


forest_clf.fit(X_train, y_train)
best_forest_clf=forest_clf.best_estimator_

In [9]:
print('random forest accuracy: ', forest_clf.best_score_)
print(best_forest_clf)
results = results.append({'clf': best_forest_clf, 'best_acc': forest_clf.best_score_}, ignore_index=True)

print('The best classifier so far is: ')
print(results.loc[results['best_acc'].idxmax()]['clf'])

random forest accuracy:  0.660848252344416
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=30,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)
The best classifier so far is: 
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=5,
                       min_wei

In [10]:
X.shape, XX.shape

((67029, 79), (28645, 79))

In [11]:
yy = results.clf.iloc[0].predict(XX)
yy.shape

(28645,)

In [13]:
submission = pd.DataFrame(list(zip(XX.VisitNumber, yy)), columns=["VisitNumber", "TripType"])


In [14]:
submission.to_csv("/content/gdrive/My Drive/Colab Notebooks/submission.csv", header=True, index=False)

Se opto por utilizar  RandomizedSearchCV debido a que mejora los tiempos de ejecución respecto a GridSearchCV. Se obtuvieron resultados superiores en  kaggle respecto al score inicial.



# **Random Forest sin grilla de parametros**

De esta manera se obtuvo un mejor score que al usar RandomizedSearchCV.


In [15]:
model=RandomForestClassifier(n_estimators=60, random_state=2, min_samples_leaf=2, min_samples_split=10)
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=60,
                       n_jobs=None, oob_score=False, random_state=2, verbose=0,
                       warm_start=False)

In [16]:
y_predicted=model.predict(XX)
model.score(X_valid, y_valid)

0.6695509473370133

In [17]:
y_predicted

array([ 9, 40, 21, ...,  3, 39, 39])

In [18]:
X.shape, XX.shape

((67029, 79), (28645, 79))

In [19]:
submission = pd.DataFrame(list(zip(XX.VisitNumber, y_predicted)), columns=["VisitNumber", "TripType"])

In [20]:
submission.to_csv("/content/gdrive/My Drive/Colab Notebooks/submission.csv", header=True, index=False)

# **Boosting**

Al usar XGBClassifier con los siguientes parametros obtuvimos los mejores resultados.


In [28]:
model_3=XGBClassifier(n_estimators=500,random_state=2, gamma=0.7, colsample_bytree=0.5, learning_rate=0.2)
model_3.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0.7,
              learning_rate=0.2, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=2,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [29]:
y_pred=model_3.predict(XX)

In [30]:
# evaluate predictions
model_3.score(X_valid, y_valid)

0.6983440250634044

In [31]:
X.shape, XX.shape


((67029, 79), (28645, 79))

In [32]:
submission = pd.DataFrame(list(zip(XX.VisitNumber, y_pred)), columns=["VisitNumber", "TripType"])

In [33]:
submission.to_csv("/content/gdrive/My Drive/Colab Notebooks/submission.csv", header=True, index=False)